In [3]:
import pandas as pd
import numpy as np

## Read in data and delete out null rows for home_tract and tract

In [27]:
%%time
sf_data = pd.read_csv("data/sf_with_homeloc.csv" , nrows = 100000)
sf_data = sf_data[~sf_data["home_tract"].isna()]
sf_data = sf_data[~sf_data["tract"].isna()]

Wall time: 233 ms


In [7]:
#Only need 3 columns
sf_data = sf_data[["date" , "tract" , "home_tract"]]

In [9]:
#read in neighbor dataset
nn_data = pd.read_csv("data/sfbay_area_nearest_neighbor.csv")

In [10]:
nn_data.head()

,OBJECTID,SRC__TRACT,NBR__TRACT,SRC_GEO_FI,NBR_GEO_FI,LENGTH,NODE_COUNT
0,1,10100,10200,6075010100,6075010200,287.614832,0
1,2,10100,10300,6075010100,6075010300,469.930769,0
2,3,10100,10400,6075010100,6075010400,868.522281,0
3,4,10100,10500,6075010100,6075010500,454.806184,0
4,5,10200,10100,6075010200,6075010100,287.614832,0


In [11]:
sf_data.head()

,date,tract,home_tract
0,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09
1,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09
2,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09
3,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09
4,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09


Create mapping of tracts to their neighbors using a dictionary

In [28]:
neighbors = {}
for src,dest in zip(nn_data["SRC_GEO_FI"] , nn_data["NBR_GEO_FI"]):
    if src not in neighbors:
        neighbors[src] = set()
    neighbors[src].add(dest)

Label tracts by appropriate names based on status

In [29]:
def processNeighbors(row):
    tract = int(row["tract"])
    home_tract = int(row["home_tract"])
    if(tract == home_tract):
        return "home"
    elif(home_tract in neighbors and tract in neighbors[home_tract]):
        return "neighbor"
    elif(home_tract in neighbors):
        return "non-neighbor"
    else:
        return "not in set?"

In [30]:
sf_data["locality"] = sf_data.apply(processNeighbors, axis=1)
sf_data["locality"].value_counts()

home            49079
non-neighbor    31636
not in set?     12105
neighbor         6904
Name: locality, dtype: int64

In [10]:
sf_data = sf_data[sf_data["locality"] !=  "not in set?"]

In [28]:
sf_data["locality"].value_counts()

home            43106
non-neighbor    28012
neighbor         6070
Name: locality, dtype: int64

In [29]:
sf_data.head()

,sf_with_homeloc.csv,u_id,lat,lon,date,tract,home_tract,locality
0,2.117882e-293,1.112537e-308,37.423851,-122.196872,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09,non-neighbor
1,2.117950e-293,1.112537e-308,38.674385,-121.342140,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09,home
2,2.117956e-293,1.112537e-308,38.393944,-121.968185,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09,home
3,2.117963e-293,1.112537e-308,37.694860,-122.484290,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09,non-neighbor
4,2.117966e-293,1.112537e-308,37.325984,-121.896728,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09,home


### Group by locality

In [30]:
overall = sf_data.groupby(["home_tract" , "locality" ]).size().reset_index().rename(columns={0:'count'})

### Groupy by time of day

In [31]:
sf_data['hour'] = sf_data['date'].str[11:13]
dayvalues = ['04', '05', '06', '07', '08', '09', '10', '11', '12', '13','14', '15', '16', '17', '18']
sf_data['day'] = sf_data.hour.map(lambda x: 1 if x in dayvalues else 0)

In [32]:
dayNight = sf_data.groupby(["home_tract" , "locality", "day"]).size().reset_index().rename(columns={0:'count'})
dayNight.head()

,home_tract,locality,day,count
0,6.001400e+09,home,0,3
1,6.001400e+09,home,1,14
2,6.001400e+09,neighbor,0,1
3,6.001400e+09,neighbor,1,1
4,6.001400e+09,non-neighbor,0,5


### Group By Weekday

In [304]:
sf_data['date_helper'] = sf_data['date'].str[0:10]
sf_data['weekday'] = sf_data.date_helper.map(lambda x: 0 if pd.to_datetime(x).weekday() > 4 else 1)


In [305]:
weekDay = sf_data.groupby(["home_tract" , "locality", "weekday"]).size().reset_index().rename(columns={0:'count'})
weekDay.head()

,home_tract,locality,weekday,count
0,6.001400e+09,home,0,3
1,6.001400e+09,home,1,13
2,6.001400e+09,neighbor,1,1
3,6.001400e+09,non-neighbor,0,2
4,6.001400e+09,non-neighbor,1,14


### From groupings with combo categories as rows convert to combo categories as cells

In [306]:
tracts = list(neighbors.keys())

In [307]:
counts = [[] , [] , []]
dayCounts = [[[], []] , [[],[]] , [[],[]]]
weekDayCounts = [[[], []] , [[],[]] , [[],[]]]
locs = ["home" , "neighbor" , "non-neighbor"]


for tract in tracts:
    one_tract = overall[overall["home_tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        if(len(one_loc) > 0):
            counts[i].append(one_loc["count"].iloc[0])
        else:
            counts[i].append(0)
        i+=1
        
for tract in tracts:
    one_tract = dayNight[dayNight["home_tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["day"] == val]
            if(len(one_val) > 0):
                dayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                dayCounts[i][val].append(0)
        i+=1
                         
for tract in tracts:
    one_tract = weekDay[weekDay["home_tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["weekday"] == val]
            if(len(one_val) > 0):
                weekDayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                weekDayCounts[i][val].append(0)
        i+=1

In [308]:
sf_final = pd.DataFrame({"tract": pd.Series(tracts)})
i=0
for loc in locs:
    sf_final[loc] = pd.Series(counts[i])
    sf_final[loc + " day"] = pd.Series(dayCounts[i][1])
    sf_final[loc + " night"] = pd.Series(dayCounts[i][0])
    sf_final[loc + " weekday"] = pd.Series(weekDayCounts[i][1])
    sf_final[loc + " weekend"] = pd.Series(weekDayCounts[i][0])
    i+=1

In [309]:
sf_final

,tract,home,home day,home night,home weekday,home weekend,neighbor,neighbor day,neighbor night,neighbor weekday,neighbor weekend,non-neighbor,non-neighbor day,non-neighbor night,non-neighbor weekday,non-neighbor weekend
0,6075010100,18,14,4,12,6,5,4,1,4,1,63,31,32,54,9
1,6075010200,24,21,3,22,2,9,5,4,9,0,32,13,19,27,5
2,6075010300,16,11,5,14,2,7,1,6,6,1,32,12,20,26,6
3,6075010400,2,2,0,2,0,3,0,3,0,3,15,8,7,13,2
4,6075010500,44,24,20,43,1,14,6,8,10,4,111,39,72,86,25
5,6075010600,24,16,8,17,7,10,5,5,10,0,49,23,26,40,9
6,6075010700,5,3,2,4,1,6,0,6,5,1,16,6,10,13,3
7,6075010800,8,7,1,6,2,3,2,1,3,0,27,6,21,24,3
8,6075010900,34,13,21,32,2,11,4,7,9,2,73,26,47,60,13
9,6075011000,21,9,12,18,3,2,1,1,2,0,27,18,9,21,6


In [310]:
#Output
sf_final.to_csv("grid.csv")